In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('FINAL DATASETS\Lillard Trade Final.csv')
df

,Username,Time,Tweet,Retweets,Likes,Interaction Count
0,LakeShowYo,2023-09-27T22:26:53.000Z,“Giannis drives… kicks it out… Dame from the l...,2500,27000,29500
1,ClutchPoints,2023-09-27T23:54:36.000Z,"""It's a perfect situation for Damian Lillard.....",97,580,677
2,Zay,2023-09-27T23:17:15.000Z,Give me this 5 over Dame & Giannis Bucks,17,145,162
3,w4yst,2023-09-27T22:36:48.000Z,DAME TO THE BUCKS NIGGA,0,5,15
4,Santi,2023-09-27T23:54:41.000Z,Heat are front runners for Beal\n\nDon’t make ...,42,178,220
...,...,...,...,...,...,...
1458,Blake Schofield,2023-09-27T20:41:37.000Z,We will see if the Bucks can get Lillard to sh...,0,1,11
1459,Derek Tadashiroll,2023-09-27T18:40:13.000Z,The Bucks are going to get Lillard to turn up ...,0,0,10
1460,Gedi,2023-09-27T19:19:16.000Z,"Permitter defense is non-existent, but it’s go...",1,0,11
1461,John Lopez,2023-09-27T18:51:39.000Z,I feel like trading holiday to get Lillard at ...,0,2,12


In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import re

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# Pretrained
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(re.sub('\n',' ',t))
    return " ".join(new_text)

def ClassifyTweets(dataframe):
    sentiment = []
    for tweet in dataframe['Tweet']:
        text = preprocess(tweet)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        sentiment.append(config.id2label[ranking[-1]])
    dataframe['Sentiment'] = sentiment
    return

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
from sklearn.metrics import accuracy_score

key = pd.read_csv('FINAL DATASETS/Lillard Final Key.csv')
pred = pd.DataFrame(key['Tweet'])
key_copy = key.copy()
fix = {0:'negative',1:'neutral',2:'positive'}
key_copy['Sentiment'] = key_copy['Sentiment'].map(fix)
key.head(2)

,Tweet,Sentiment,Subject
0,"A better fit than Damian lillard, make it HAPPEN",1,0
1,Dame + Giannis is going to be INSANE to watch....,2,0


In [4]:
#ClassifyTweets(pred)

In [5]:
#accuracy_score(key_copy['Sentiment'].values,pred['Sentiment'].values)

In [6]:
import datasets
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split as tts
key = pd.read_csv('FINAL DATASETS/Lillard Final Key.csv')

X_train, X_test, y_train, y_test = tts(key['Tweet'], key['Sentiment'], test_size = 1/6, random_state = 42)
train = pd.DataFrame(zip(X_train, y_train), columns = ['text','label'])
test = pd.DataFrame(zip(X_test, y_test), columns = ['text','label'])

train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", max_length = 514,truncation=True)

tokenized_train = train.map(tokenize_function, batched = True)
tokenized_test = test.map(tokenize_function, batched = True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir = 'Test_trainer',num_train_epochs=5)

In [8]:
import evaluate

metric = evaluate.load('accuracy')

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits,axis=1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    return accuracy

In [10]:
from transformers import Trainer

trainer = Trainer(model=model,
                 args=training_args,
                 train_dataset=tokenized_train,
                 eval_dataset=tokenized_test,
                 compute_metrics = compute_metrics)

trainer.train()

Step,Training Loss


TrainOutput(global_step=315, training_loss=0.3311675540984623, metrics={'train_runtime': 5371.4303, 'train_samples_per_second': 0.465, 'train_steps_per_second': 0.059, 'total_flos': 660353011290000.0, 'train_loss': 0.3311675540984623, 'epoch': 5.0})

In [11]:
# hopefully less than 0.245 for loss, over 70% accuracy eval
trainer.evaluate()

{'eval_loss': 1.6794116497039795,
 'eval_accuracy': 0.72,
 'eval_runtime': 96.5774,
 'eval_samples_per_second': 1.035,
 'eval_steps_per_second': 0.135,
 'epoch': 5.0}

In [15]:
trainer.save_model(r'C:\Users\Owner\Desktop\NLPprojectData\HuggingFace Models\5 epochs')